# Combine Hege-Beate's data into one file

This replaces the original download of 4xCO2 data using `netcdf-scm`. Hege-Beate's data contains many more models.

In [ ]:
import pandas as pd
import glob

In [ ]:
available_files = glob.glob('../data/cmip6-hbf/cmip_data/*/abrupt-4xCO2/*_abrupt-4xCO2_*_anomalies.txt')

In [ ]:
models = []  # not unique
runs = []
lines = []
for file in available_files:
    model = file.split('/')[4]
    run = file.split('/')[6].split('_')[2]
    models.append(model)
    runs.append(run)
    df = pd.read_csv(file, index_col=0)
    vars = {}
    for var in ['tas', 'rlut', 'rsut', 'rsdt']:
        vars[var] = df[var].values[:150].squeeze()
        line = ['CMIP', model, run, 'CMIP6', 'unspecified', 'World', 'abrupt-4xCO2', 'W m^-2', var]
        line.extend(vars[var])
        lines.append(line)
    vars['rndt'] = vars['rsdt'] - vars['rsut'] - vars['rlut']
    line = ['CMIP', model, run, 'CMIP6', 'unspecified', 'World', 'abrupt-4xCO2', 'W m^-2', 'rndt']
    line.extend(vars['rndt'])
    lines.append(line)

In [ ]:
df = pd.DataFrame(
    lines, columns = (
        ['activity_id', 'climate_model', 'member_id', 'mip_era', 'model', 'region', 'scenario', 'unit', 'variable'] +
        ["X%d" % year for year in range(1850, 2000)]
    )
)
df.dropna(inplace=True)

In [ ]:
df.to_csv('../data/cmip6-hbf/4xCO2.csv', index=False)